# Lab 4: Regular Expressions and Factors


Derek Hansen

Wednesday, November 4, 2020
1:00PM-2:30PM

# Preliminaries



# Regular Expressions


In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Regular Expressions are Hard

-  Even seasoned programmers often struggle with regular expressions; they require a lot of practice to master.
-  Many people have written entire books with different animals on the cover just on the topic, and this is just from one publisher.


![owl](owl.jpg)
![weasel](weasel.jpg)
![bat](bat.jpg)


-  This website will let you test out regular expressions on the fly: https://www.regexpal.com/
-  Be sure to check the "multiline" box under "flags"

![](regex_pal2.png)

## RegExr
-  RegExr is a more powerful website that color-codes expressions as you build them: https://regexr.com/

![](regexr.png)

## Using Regular Expressions in dplyr
- Regular expressions aren't just useful for text data. It is also useful when data comes in a wide format with many columns that are hard to reconcile.
- Here we a table of crime data by age, sex, and race from the ICSPR at the University of Michigan (original source here: https://www.icpsr.umich.edu/web/NACJD/studies/36115)
- As you'll see below, each row corresponds to a particular agency, month, and offense. The subsequent columns denotes counts of gender-age crimes (e.g. "F20" is how many reports for females age 20), or of different racial groups which are broken up by juvenile-status and race ("AW" is Adult White).
- While high-quality, this data is not very tidy; it is good example of how data actually comes in the real world.


In [4]:
icpsr_raw <- read_csv("icpsr_raw.csv")


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  ORI = col_character(),
  GROUP = col_character(),
  CORE = col_character(),
  AGENCY = col_character(),
  STNAME = col_character(),
  OFFENSE = col_character()
)
ℹ Use `spec()` for the full column specifications.




In [7]:
glimpse(icpsr_raw)

Rows: 58,277
Columns: 89
$ ASR_ID   <dbl> 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, …
$ CONTENTS <dbl> 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, …
$ STATE    <dbl> 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, …
$ ORI      <chr> "MI01001", "MI01001", "MI01001", "MI01001", "MI01001", "MI01…
$ GROUP    <chr> "8D", "8D", "8D", "8D", "8D", "8D", "8D", "8D", "8D", "8D", …
$ DIV      <dbl> 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, …
$ AGHEADER <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ YEAR     <dbl> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, …
$ MSA      <dbl> 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, …
$ COUNTY   <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ SEQNO    <dbl> 70480, 70480, 70480, 70480, 70480, 70480, 70480, 70480, 7048…
$ SUB      <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ CORE     <chr> "N", "N", 

- We want to form a table of the crime data of gender and age groups. We use regular expressions to select the columns that match the age-gender format we saw
- We can print out our columns, then copy-pasted them into RegExr

In [17]:
message(paste0(names(icpsr_raw), collapse="\n"))

ASR_ID
CONTENTS
STATE
ORI
GROUP
DIV
AGHEADER
YEAR
MSA
COUNTY
SEQNO
SUB
CORE
COVBY
POP
AGENCY
STNAME
MONTH
MOHEADER
BREAK
AREO
ZERO
DTLASTUP
DTPRUP1
DTPRUP2
JUVDISP
JDHANDDP
JDREFJC
JDREFWA
JDREFOPA
JDREFCC
OFFENSE
OCCUR
M0_9
M10_12
M13_14
M15
M16
M17
M18
M19
M20
M21
M22
M23
M24
M25_29
M30_34
M35_39
M40_44
M45_49
M50_54
M55_59
M60_64
M65
F0_9
F10_12
F13_14
F15
F16
F17
F18
F19
F20
F21
F22
F23
F24
F25_29
F30_34
F35_39
F40_44
F45_49
F50_54
F55_59
F60_64
F65
AW
AB
AI
AA
JW
JB
JI
JA
AH
AN
JH
JN



![](gender_age2.png)

-  We then use the ```matches``` function within ```select```. ```matches``` will keep all columns that match our regular expression.
-  Matches works with tidyr functions as well

In [8]:
icpsr_gender_age <- icpsr_raw %>%
  filter(CONTENTS == 3) %>%
  select(originating_agency = ORI,
         month = MONTH,
         offense = OFFENSE,
         matches("^(M|F)[0-9]+")) %>%
  gather(matches("^(M|F)[0-9]+"), 
         key = "gender_age",
         value = "count") %>%
  mutate(count = ifelse(count == 99999, 0, count)) %>%
  mutate(count = ifelse(count == 99998, NA_real_, count)) %>%
  separate(gender_age, into = c("gender", "age"), sep=1)

In [10]:
print(icpsr_gender_age)

# A tibble: 2,521,640 x 6
   originating_agency month offense gender age   count
   <chr>              <dbl> <chr>   <chr>  <chr> <dbl>
 1 MI01001                1 26      M      0_9       0
 2 MI01001                2 26      M      0_9       0
 3 MI01001                3 18      M      0_9       0
 4 MI01001                3 185     M      0_9       0
 5 MI01001                3 18F     M      0_9       0
 6 MI01001                4 18      M      0_9       0
 7 MI01001                4 180     M      0_9       0
 8 MI01001                4 18D     M      0_9       0
 9 MI01001                5 18      M      0_9       0
10 MI01001                5 185     M      0_9       0
# … with 2,521,630 more rows


- In this simple case, we could just use the tidyr ```separate``` function with an index (1) 
- The ```extract``` function from ```tidyr``` is more powerful, as we can have it search for particular patterns

In [25]:
icpsr_gender_age <- icpsr_raw %>%
  filter(CONTENTS == 3) %>%
  select(originating_agency = ORI,
         month = MONTH,
         offense = OFFENSE,
         matches("^(M|F)[0-9]+")) %>%
  gather(matches("^(M|F)[0-9]+"), 
         key = "gender_age",
         value = "count") %>%
  mutate(count = ifelse(count == 99999, 0, count)) %>%
  mutate(count = ifelse(count == 99998, NA_real_, count)) %>%
  extract(gender_age, into = c("gender", "age"), regex=("(M|F)([0-9]+)"))
print(icpsr_gender_age )

# A tibble: 2,521,640 x 6
   originating_agency month offense gender age   count
   <chr>              <dbl> <chr>   <chr>  <chr> <dbl>
 1 MI01001                1 26      M      0         0
 2 MI01001                2 26      M      0         0
 3 MI01001                3 18      M      0         0
 4 MI01001                3 185     M      0         0
 5 MI01001                3 18F     M      0         0
 6 MI01001                4 18      M      0         0
 7 MI01001                4 180     M      0         0
 8 MI01001                4 18D     M      0         0
 9 MI01001                5 18      M      0         0
10 MI01001                5 185     M      0         0
# … with 2,521,630 more rows


-  What happened? Try looking back at our website code
-  Our regular expression was good enough to find the right columns, but "[0-9]+" doesn't match to any of the names with underscores (such as "M0_9")
-  This is why regular expressions are tricky to master!

In [29]:
icpsr_gender_age <- icpsr_raw %>%
  filter(CONTENTS == 3) %>%
  select(originating_agency = ORI,
         month = MONTH,
         offense = OFFENSE,
         matches("^(M|F)[0-9]+")) %>%
  gather(matches("^(M|F)[0-9]+"), 
         key = "gender_age",
         value = "count") %>%
  mutate(count = ifelse(count == 99999, 0, count)) %>%
  mutate(count = ifelse(count == 99998, NA_real_, count)) %>%
  extract(gender_age, into = c("gender", "age"), regex=("(M|F)([0-9]+$|[0-9]+_[0-9]+)"))
print(icpsr_gender_age )

# A tibble: 2,521,640 x 6
   originating_agency month offense gender age   count
   <chr>              <dbl> <chr>   <chr>  <chr> <dbl>
 1 MI01001                1 26      M      0_9       0
 2 MI01001                2 26      M      0_9       0
 3 MI01001                3 18      M      0_9       0
 4 MI01001                3 185     M      0_9       0
 5 MI01001                3 18F     M      0_9       0
 6 MI01001                4 18      M      0_9       0
 7 MI01001                4 180     M      0_9       0
 8 MI01001                4 18D     M      0_9       0
 9 MI01001                5 18      M      0_9       0
10 MI01001                5 185     M      0_9       0
# … with 2,521,630 more rows


**Exercise:**

Instead of age being a character, split the range into age_min and age_max. If there is just one age, set age_min=age_max

**Solution:**

You could just do ```separate``` on the "age" column. Or you can do it directly, all with regular expressions!

We need to make sure that we handle all cases properly. We define three groups to define gender_age:
-  ```"(M|F)"``` matches either male or female
-  ```"([0-9]+)"``` matches "9", "20", etc.
-  ```"((_[0-9]+|)"``` matches ```"_10"```, ```"_100"```, **or nothing**. If we don't specify this, the rows where the string does not match this pattern will return "NA" for all values.

In [77]:
icpsr_gender_age <- icpsr_raw %>%
  filter(CONTENTS == 3) %>%
  select(originating_agency = ORI,
         month = MONTH,
         offense = OFFENSE,
         matches("^(M|F)[0-9]+")) %>%
  gather(matches("^(M|F)[0-9]+"), 
         key = "gender_age",
         value = "count") %>%
  mutate(count = ifelse(count == 99999, 0, count)) %>%
  mutate(count = ifelse(count == 99998, NA_real_, count)) %>%
  extract(gender_age, into = c("gender", "age_min_str", "age_max_str"), regex=("(M|F)([0-9]+)(_[0-9]+|)")) %>%
  mutate(age_min = as.integer(age_min_str), age_max=as.integer(str_sub(age_max_str, 2))) %>%
  mutate(age_max = ifelse(is.na(age_max), age_min, age_max))
print(icpsr_gender_age)


# A tibble: 2,521,640 x 9
   originating_age… month offense gender age_min_str age_max_str count age_min
   <chr>            <dbl> <chr>   <chr>  <chr>       <chr>       <dbl>   <int>
 1 MI01001              1 26      M      0           _9              0       0
 2 MI01001              2 26      M      0           _9              0       0
 3 MI01001              3 18      M      0           _9              0       0
 4 MI01001              3 185     M      0           _9              0       0
 5 MI01001              3 18F     M      0           _9              0       0
 6 MI01001              4 18      M      0           _9              0       0
 7 MI01001              4 180     M      0           _9              0       0
 8 MI01001              4 18D     M      0           _9              0       0
 9 MI01001              5 18      M      0           _9              0       0
10 MI01001              5 185     M      0           _9              0       0
# … with 2,521,630 more ro

In [78]:
print(filter(icpsr_gender_age, age_min==20))

# A tibble: 114,620 x 9
   originating_age… month offense gender age_min_str age_max_str count age_min
   <chr>            <dbl> <chr>   <chr>  <chr>       <chr>       <dbl>   <int>
 1 MI01001              1 26      M      20          ""              0      20
 2 MI01001              2 26      M      20          ""              0      20
 3 MI01001              3 18      M      20          ""              0      20
 4 MI01001              3 185     M      20          ""              0      20
 5 MI01001              3 18F     M      20          ""              0      20
 6 MI01001              4 18      M      20          ""              0      20
 7 MI01001              4 180     M      20          ""              0      20
 8 MI01001              4 18D     M      20          ""              0      20
 9 MI01001              5 18      M      20          ""              0      20
10 MI01001              5 185     M      20          ""              0      20
# … with 114,610 more rows, 